# <center>Laboratorium Analiza i bazy danych </center>

## <center>Łączenie tabel, podzapytania i funkcje agregujące</center>

## Przykładowe tabele obrazujące łączenie

Do zobrazowania operacji łączenia zostaną użyte tabele:

```sql
CREATE TABLE shape_a (
    id INT PRIMARY KEY,
    shape VARCHAR (100) NOT NULL
);
 
CREATE TABLE shape_b (
    id INT PRIMARY KEY,
    shape VARCHAR (100) NOT NULL
);
```
 
Polecenie CREATE TABLE tworzy tabelę o zadanej nazwie i strukturze. Ogólna postać to:
```sql
CREATE TABLE tab_name (
    col_name1 data_type constrain,
    col_name1 data_type constrain,
    ...
);
```
Należy uzupełnić ją danymi:
```sql
INSERT INTO shape_a (id, shape)
VALUES
    (1, 'Trójkąt'),
    (2, 'Kwadrat'),
    (3, 'Deltoid'),
    (4, 'Traper');
 
INSERT INTO shape_b (id, shape)
VALUES
    (1, 'Kwadrat'),
    (2, 'Trójkąt'),
    (3, 'Romb'),
    (4, 'Równoległobok');
```
Komenda INSERT INTO pozwala na dodanie do tabeli rekordów. Ogólna postać to:

```sql
INSERT INTO tab_name (col1_name, col2_name2, ...) 
VALUES
    (val1_col1, val2_col2),
    (val2_col1, val2_col2),
    ...
```

## Inner join 

Jest to podstawowy rodzaj złączenie. Ten sposób złączenia wybiera  te wiersze, dla których warunek złączenia jest spełniony. W żadnej z łączonych tabel kolumna użyta do łączenia nie może mieć wartości NULL. 

#### Przykład:
```sql
SELECT
    a.id id_a,
    a.shape shape_a,
    b.id id_b,
    b.shape shape_b
FROM
    shape_a a
INNER JOIN shape_b b ON a.shape = b.shape;
```
W zapytaniu powyżej użyto *aliasów* nazw tabel i column wynikowych, jest to szczególnie przydatne przy długich nazwach tabel i wprowadza czytelność w zapytaniu.

#### Wynik:
|id_a|shape_a|id_b|shape_b|
|-|-|-|-|
|1|Trójkąt|2|Trójkąt|
|2|Kwadrat|1|Kwadrat|

## OUTER JOIN

Istnieją trzy rodzaje złączeń OUTER:
- LEFT OUTER JOIN,
- RIGHT OUTER JOIN,
- FULL OUTER JOIN.

### LEFT OUTER JOIN

Ten rodzaj złączenie zwróci wszystkie rekordy z lewej tablicy i dopasuje do nich rekordy z prawej tablicy które spełniją zadany warunek złączenia. Jeżeli w prawej tablicy nie występują rekordy spełnijące warunek złączenia z lewą w ich miejscu pojawią się wartości NULL.

#### Przykład 1:
```sql
SELECT
    a.id id_a,
    a.shape shape_a,
    b.id id_b,
    b.shape shape_b
FROM
    shape_a a
LEFT JOIN shape_b b ON a.shape = b.shape;
```
#### Wynik:
|id_a|shape_a|id_b|shape_b|
|-|-|-|-|
|1|Trójkąt|2|Trójkąt|
|2|Kwadrat|1|Kwadrat|
|3|Deltoid|NULL|NULL|
|4|Traper|NULL|NULL|

#### Przykład 2:
```sql
SELECT
    b.id id_b,
    b.shape shape_b,
    a.id id_a,
    a.shape shape_a   
FROM
    shape_b b
LEFT JOIN shape_a a ON a.shape = b.shape;
```
#### Wynik:
|id_a|shape_a|id_b|shape_b|
|-|-|-|-|
|1|Kwadrat|2|Kwadrat|
|2|Trójkąt|1|Trójkąt|
|3|Romb|NULL|NULL|
|4|Równoległobok|NULL|NULL|

### RIGHT OUTER JOIN

Działa jak left outer join z tym, że prawa tablica w zapytaniu jest brana w całości.

#### Przykład:
```sql
SELECT
    a.id id_a,
    a.shape shape_a,
    b.id id_b,
    b.shape shape_b
FROM
    shape_a a
RIGHT JOIN shape_b b ON a.shape = b.shape;
```

#### Wynik:
|id_a|shape_a|id_b|shape_b|
|-|-|-|-|
|2|Kwadrat|1|Kwadrat|
|1|Trójkąt|2|Trójkąt|
|NULL|NULL|3|Romb|
|NULL|NULL|4|Równoległobok|


### FULL OUTER JOIN

Jest złączeniem które zwraca:
- wiersze dla których warunek złączenia jest spełniony,
- wiersze z lewej tabeli dla których nie ma odpowiedników w prawej,
- wiersze z prawej tabeli dla których nie ma odpowiedników w lewej. 

#### Przykład:
```sql
SELECT
    a.id id_a,
    a.shape shape_a,
    b.id id_b,
    b.shape shape_b
FROM
    shape_a a
FULL JOIN shape_b b ON a.shape = b.shape;
```
|id_a|shape_a|id_b|shape_b|
|-|-|-|-|
|1|Trójkąt|2|Trójkąt|
|2|Kwadrat|1|Kwadrat|
|3|Deltoid"|NULL|NULL|
|4|Traper|NULL|NULL|
|NULL|NULL|3|Romb|
|NULL|NULL|4|Równoległobok|

## Podzapytania

Podzapytanie zagnieżdżone SELECT znajduje się wewnątrz zewnętrznego zapytania SELECT, np. po klauzuli WHERE, HAVING lub FROM. W przypadku tego rodzaju zapytań w pierwszej kolejności wykonywane są wewnętrzne zapytania SELECT, a ich wynik jest wykorzystywany do zewnętrznego zapytania SELECT. Stąd łatwo zuważyć, że mogą one służyć do poprawy wydajności obsługi zapytania. Należy dobierać podzapytania tak by najbardziej zagnieżdżone podzapytanie zawierało najmniejszy zbiór poszukiwań. 

#### Przykład:
Jeżeli chcemy znaleźć w bazie informację o tytułach filmów zwróconych w zadanym okresie możemy wykonać następujące zapytanie:
```sql
SELECT
   film_id,
   title
FROM
   film
WHERE
   film_id IN (
      SELECT
         inventory.film_id
      FROM
         rental
      INNER JOIN inventory ON inventory.inventory_id = rental.inventory_id
      WHERE
         return_date BETWEEN '2005-05-29'
      AND '2005-05-30'
   );
```

#### Wynik
|film_id|title|
|-|-|
|307|Fellowship Autumn|
|255|Driving Polish|
|388|Gunfight Moon|
|130|Celebrity Horn|
|563|Massacre Usual|
|397|Hanky October|
|...|...|

### Używanie podzapytań

Pod zapytania mogą być używane w :
- SELECT,
- UPDATE,
- DELETE,
- Funkcjach agregujących,
- Do definiowania tabel tymczasowych.

Używając podzapytań zapytania SQL szybko mogą stać się mało czytelne. Przez co będą trudne w zrozumieniu i późniejszym utrzymaniu. W celu analizy zapytań można użyć klauzuli __EXPLAIN__, która przeanalizuje zapytanie. Klauzula ta może służyć również do porównywania wydajności zapytań

#### Przykład:
```sql
EXPLAIN SELECT
   *
FROM
   film
```

## Funkcje agregujące

Funkcje agregujące wykonują obliczenia na zestawie wierszy i zwracają pojedynczy wiersz. PostgreSQL udostępnia wszystkie standardowe funkcje agregujące SQL w następujący sposób:
- AVG () - zwraca średnią wartość.
- COUNT () - zwraca liczbę wartości.
- MAX () - zwraca maksymalną wartość.
- MIN () - zwraca minimalną wartość.
- SUM () - zwraca sumę wszystkich lub różnych wartości.

Pełna lista funkcji agregującej: https://www.postgresql.org/docs/9.5/functions-aggregate.html

Często używamy funkcji agregujących z klauzulą GROUP BY w instrukcji SELECT. W tych przypadkach klauzula GROUP BY dzieli zestaw wyników na grupy wierszy i funkcja agregująca wykonuje obliczenia dla każdej grupy, np. maksimum, minimum, średnia itp. Funkcji agregujących można używać funkcji agregujących jako wyrażeń tylko w następujących klauzulach: SELECT i HAVING.

### GROUP BY
Klauzula GROUP BY dzieli wiersze zwrócone z instrukcji SELECT na grupy. Dla  każdej grupy można zastosować funkcję agregującą, np. SUM aby obliczyć sumę pozycji lub
COUNT aby uzyskać liczbę elementów w grupach.

Poniższa instrukcja ilustruje składnię klauzuli GROUP BY:
```sql
SELECT 
    column_1, 
    aggregate_function(column_2)
FROM 
    tbl_name
GROUP BY 
    column_1;
```
Klauzula GROUP BY musi pojawić się zaraz po klauzuli FROM lub WHERE, n0astępnie GROUP BY zawiera listę  kolumna oddzielonych przecinkami. 

### HAVING
Często używamy klauzuli HAVING w połączeniu z klauzulą GROUP BY do filtrowania wierszy grup
które nie spełniają określonego warunku.

Poniższa instrukcja ilustruje typową składnię klauzuli HAVING:
```sql
SELECT
    column_1,
    aggregate_function (column_2)
FROM
    tbl_name
GROUP BY
    column_1
HAVING
    condition;
```
Klauzula HAVING ustawia warunek dla wierszy grup utworzonych przez klauzulę GROUP BY.  

Klauzula GROUP BY ma zastosowanie, podczas gdy klauzula WHERE określa wcześniej warunki dla poszczególnych wierszy.

## Zadania wprowadzające
Wykonaj zapytania przy użyciu DBMS:  
  
1. Znajdź listę wszystkich filmów o tej samej długości.
2. Znajdź wszystkich klientów mieszkających w tym samym mieście.
3. Oblicz średni koszt wypożyczenia wszystkich filmów.
4. Oblicz i wyświetl liczbę filmów we wszystkich kategoriach.
5. Wyświetl liczbę wszystkich klientów pogrupowanych według kraju.
6. Wyświetl informacje o sklepie, który ma więcej niż 100 klientów i mniej niż 300 klientów.
7. Wybierz wszystkich klientów, którzy oglądali filmy ponad 200 godzin.
8. Oblicz średnią wartość wypożyczenia filmu.
9. Oblicz średnią wartość długości filmu we wszystkich kategoriach.
10. Znajdź najdłuższe tytuły filmowe we wszystkich kategoriach.
11. Znajdź najdłuższy film we wszystkich kategoriach. Porównaj wynik z pkt 10.

## Zadanie 1
Znajdź listę wszystkich filmów o tej samej długości.

In [160]:
import psycopg2 as pg
import pandas as pd

connection = pg.connect(host='pgsql-196447.vipserv.org', port=5432, dbname='wbauer_adb', user='wbauer_adb', password='adb2020')
df = pd.read_sql('select length, title from film ORDER BY length',con=connection)
df


,length,title
0,46,Ridgemont Submarine
1,46,Iron Moon
2,46,Alien Center
3,46,Kwai Homeward
4,46,Labyrinth League
...,...,...
995,185,Darn Forrester
996,185,Sweet Brotherhood
997,185,Soldiers Evolution
998,185,Worst Banger


## Zadanie 2
Znajdź wszystkich klientów mieszkających w tym samym mieście.

In [161]:
df = pd.read_sql('select first_name, last_name, ci.city from customer c \
        inner join address a on a.address_id = c.address_id\
        inner join city ci on ci.city_id = a.city_id\
        ORDER BY ci.city',con=connection)
df

,first_name,last_name,city
0,Julie,Sanchez,A Corua (La Corua)
1,Peggy,Myers,Abha
2,Tom,Milner,Abu Dhabi
3,Glen,Talbert,Acua
4,Larry,Thrasher,Adana
...,...,...,...
594,Constance,Reid,Zaria
595,Jack,Foust,Zeleznogorsk
596,Byron,Box,Zhezqazghan
597,Guy,Brownlee,Zhoushan


## Zadanie 3
Oblicz średni koszt wypożyczenia wszystkich filmów.

In [162]:
df = pd.read_sql('select title, replacement_cost from film',con=connection)
df

,title,replacement_cost
0,Chamber Italian,14.99
1,Grosse Wonderful,19.99
2,Airport Pollock,15.99
3,Bright Encounters,12.99
4,Academy Dinosaur,20.99
...,...,...
995,Young Language,9.99
996,Youth Kick,14.99
997,Zhivago Core,10.99
998,Zoolander Fiction,28.99


## Zadanie 4
Oblicz i wyświetl liczbę filmów we wszystkich kategoriach.

In [163]:
df = pd.read_sql('select c.name, count(title) from film f\
        inner join film_category fc on fc.film_id = f.film_id\
        inner join category c on c.category_id = fc.category_id\
        GROUP BY c.category_id\
        ORDER BY c.name',con=connection)
df

,name,count
0,Action,64
1,Animation,66
2,Children,60
3,Classics,57
4,Comedy,58
5,Documentary,68
6,Drama,62
7,Family,69
8,Foreign,73
9,Games,61


## Zadanie 5
Wyświetl liczbę wszystkich klientów pogrupowanych według kraju.

In [164]:
df = pd.read_sql('select country, count(country) from customer c\
        inner join address a on a.address_id = c.address_id\
        inner join city ci on ci.city_id = a.city_id\
        inner join country co on co.country_id = ci.country_id\
        GROUP BY co.country\
        ORDER by country',con=connection)
df

,country,count
0,Afghanistan,1
1,Algeria,3
2,American Samoa,1
3,Angola,2
4,Anguilla,1
...,...,...
103,Vietnam,6
104,"Virgin Islands, U.S.",1
105,Yemen,4
106,Yugoslavia,2


## Zadanie 6
Wyświetl informacje o sklepie, który ma więcej niż 100 klientów i mniej niż 300 klientów.

In [165]:
df = pd.read_sql('select s.address_id, count(s.address_id) AS count from store s\
        inner join customer cu on cu.store_id = s.store_id\
        GROUP BY s.address_id\
        HAVING count(s.address_id) > 100 AND count(s.address_id) < 300',con=connection)
df

,address_id,count
0,2,273


## Zadanie 7
Wybierz wszystkich klientów, którzy oglądali filmy ponad 200 godzin.

In [166]:
df = pd.read_sql('select c.first_name, c.last_name, sum(length) AS sum from customer c\
        inner join rental r on r.customer_id = c.customer_id\
        inner join inventory i on i.inventory_id = r.inventory_id\
        inner join film f on f.film_id = i.film_id\
        GROUP BY c.customer_id\
        HAVING sum(length) > 200',con=connection)
df

,first_name,last_name,sum
0,Vickie,Brewer,3484
1,Connie,Wallace,2795
2,Sylvia,Ortiz,3567
3,Miriam,Mckinney,2672
4,Gwendolyn,May,2890
...,...,...,...
594,Jeffrey,Spear,2678
595,Hilda,Hopkins,3080
596,Clara,Shaw,4808
597,Maria,Miller,3494


## Zadanie 8
Oblicz średnią wartość wypożyczenia filmu.

In [167]:
df = pd.read_sql('select avg(replacement_cost) from film',con=connection)
df

,avg
0,19.984


## Zadanie 9
Oblicz średnią wartość długości filmu we wszystkich kategoriach.

In [168]:
df = pd.read_sql('select c.name, avg(length) from film f\
        inner join film_category fc on fc.film_id = f.film_id\
        inner join category c on c.category_id = fc.category_id\
        GROUP BY c.category_id\
        ORDER BY c.name',con=connection)
df

,name,avg
0,Action,111.609375
1,Animation,111.015152
2,Children,109.800000
3,Classics,111.666667
4,Comedy,115.827586
5,Documentary,108.750000
6,Drama,120.838710
7,Family,114.782609
8,Foreign,121.698630
9,Games,127.836066


## Zadanie 10
Znajdź najdłuższe tytuły filmowe we wszystkich kategoriach.

In [169]:
df = pd.read_sql('SELECT fi.title, t.name, t.max FROM film fi\
        INNER JOIN film_category f_c ON f_c.film_id = fi.film_id\
        INNER JOIN (SELECT c.name, c.category_id, max(LENGTH(f.title)) from film f\
        INNER JOIN film_category fc ON fc.film_id = f.film_id\
        INNER JOIN category c ON c.category_id = fc.category_id\
        GROUP BY c.category_id) as t\
        ON t.category_id = f_c.category_id AND LENGTH(fi.title) = max\
        ORDER BY t.name', con=connection)
df

,title,name,max
0,Entrapment Satisfaction,Action,23
1,Telemark Heartbreakers,Animation,22
2,Heartbreakers Bright,Children,20
3,Microcosmos Paradise,Children,20
4,Sweethearts Suspects,Children,20
5,Extraordinary Conquerer,Classics,23
6,Trainspotting Strangers,Comedy,23
7,Intolerable Intentions,Documentary,22
8,Deliverance Mulholland,Documentary,22
9,Goldfinger Sensibility,Drama,22


## Zadanie 11
Znajdź najdłuższy film we wszystkich kategoriach

In [170]:
df = pd.read_sql('select fi.title, t.name, t.max from film fi\
        inner join film_category f_c on f_c.film_id = fi.film_id\
        inner join (select c.category_id, c.name, max(f.length) from film f\
        inner join film_category fc on fc.film_id = f.film_id\
        inner join category c on c.category_id = fc.category_id\
        GROUP BY c.category_id) t\
        ON t.category_id = f_c.category_id and fi.length = t.max\
        ORDER BY t.name', con=connection)
df

,title,name,max
0,Darn Forrester,Action,185
1,Worst Banger,Action,185
2,Pond Seattle,Animation,185
3,Gangs Pride,Animation,185
4,Fury Murder,Children,178
5,Wrong Behavior,Children,178
6,Conspiracy Spirit,Classics,184
7,Control Anthem,Comedy,185
8,Young Language,Documentary,183
9,Wife Turn,Documentary,183


## Zadanie implementacyjne
Zaimplementuj wszystkie funkcje w pliku main.py zgodnie z opisem a następnie przetestuj je w notatniku.

,city
0,A Corua (La Corua)
1,Abha
2,Abu Dhabi
3,Acua
4,Adana
...,...
595,Zaria
596,Zeleznogorsk
597,Zhezqazghan
598,Zhoushan
